In [59]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
import os
import pickle

In [60]:
# display all columns
pd.set_option('display.max_columns', None)

# Import Data

In [61]:
# access folder where data is downloaded
data_download_file_path = './data/data_download'
dir_list = os.listdir(data_download_file_path)

In [62]:
# create empty dataframe to append to
df = pd.DataFrame()

# create master dataframe
for file in dir_list:
    file_path = os.path.join(data_download_file_path, file)
    # open each zip file
    with ZipFile(file_path, 'r') as zip:
        json_file = [i for i in zip.namelist() if '.json' in i][0]
        # read json as a dataframe
        df_year = pd.read_json(zip.open(json_file))
        # add to master_df
        df = pd.concat([df, df_year])

In [63]:
# save dataframe to pickle
df.to_pickle('./data/aviation_2002_2022_master_data')

# Data Cleaning

In [64]:
# convert columns to lowercase
df.columns = [col.lower() for col in df.columns]

In [65]:
# examine data shape
df.shape

(37601, 52)

There are three columns which contain text narratives (descriptions) of the aviation accident. We will keep `analysisnarrative` and `factualnarrative` which have the least missing values. We will drop `prelimnarrative`

In [66]:
# drop prelimnarrative
df.drop(columns='prelimnarrative', inplace=True)

Since we will be conducting NLP using the narrative columns, we will only keep rows where both `analysisnarrative` and `factualnarrative` are not null

In [67]:
# analysisnarrative and factualnarrative null counts
df['analysisnarrative'].isnull().sum(), df['factualnarrative'].isnull().sum()

(4319, 5833)

In [68]:
# drop where factualnattative and analysisnarrative are null
df = df.loc[(~df['analysisnarrative'].isnull())&(~df['factualnarrative'].isnull())].copy()

We will now examine the data for nulls

In [69]:
def show_null_counts(df: pd.DataFrame) -> pd.DataFrame:
    """
    displays the non-zero null counts of columns in a dataframe
    """
    nulls = df.isna().sum().to_frame()
    nulls.columns = ['null_ct']
    return nulls.loc[nulls['null_ct']!=0].sort_values(by='null_ct', ascending=False)

In [70]:
len(df)

31731

In [71]:
show_null_counts(df)

,null_ct
regulatortype,31731
pipelineoperator,31731
hazmattype,31731
hazmatoperator,31731
hazardclass,31731
dotcontainerspec,31731
containertype,31731
competentauthority,31731
chemicalreleased,31731
stateofmaterialother,31731


We will drop all columns which have a null count of 30_000 or more

In [72]:
df.drop(columns=['cm_recentreportpublishdate',
                 'chemicalreleased',
                 'competentauthority',
                 'containertype',
                 'dotcontainerspec',
                 'hazardclass',
                 'hazmatoperator',
                 'hazmattype',
                 'specialpermit',
                 'stateofmaterial',
                 'stateofmaterialother',
                 'subhazardclass',
                 'pipelineoperator',
                 'pipelinetype',
                 'regulatortype',
                 'cm_reportnum',
                 'cm_boardmeetingdate',
                 'cm_docketdate'],
        inplace=True)

Make sure data is limited to `cm_closed` values of "True" to only look at accident cases which have been closed

In [73]:
# examine value counts for cm_closed
df['cm_closed'].value_counts().to_frame()

,cm_closed
True,31731


In [74]:
# confirm that the cm_mkey is primary key/ unique
df['cm_mkey'].nunique(), len(df)

(31731, 31731)

After only keeping cases which are open, drop columns `cm_closed` and `cm_completionstatus` as they will not be used in model

In [75]:
# don't need cm_closed and cm_completionstatus
df.drop(columns=['cm_closed', 'cm_completionstatus'], inplace=True)

Make sure data limited to aviation accidents

In [76]:
# look at values for cm_mode
list(df['cm_mode'].unique())

['Aviation']

We will only examine cases within the USA since we are looking to influence policy-makers within the U.S.

In [77]:
# drop non-USA cases
df = df.loc[df['cm_country']=='USA'].copy()

In [78]:
len(df)

28599

In [79]:
# examine null counts again
show_null_counts(df)

,null_ct
cm_launch,18486
cm_docketoriginalpublishdate,10391
airportid,9081
airportname,8849
cm_reportdate,1824
cm_agency,1192
cm_longitude,416
cm_latitude,415
cm_state,108
cm_highestinjury,101


#### `cm_vehicles`
`cm_vehicles` is stored as a json within a list, so we will convert the information to columns in the dataframe

In [80]:
# reset index
df.reset_index(drop=True, inplace=True)
# json is stored in list - create new column to extract json from list
df['cm_vehicles_first'] = df['cm_vehicles'].str[0]
# normalize json
df2 = pd.json_normalize(df['cm_vehicles_first'])
# reset index
df2.index = df.index
# join normalized json
# "avia" short for "aviation"
avia = df.join(df2)

del df
del df2

In [81]:
# examine shape of joined dataframe
avia.shape

(28599, 58)

In [82]:
# drop cm_vehicles_first and cm_vehicles since the information was extracted
avia.drop(columns=['cm_vehicles_first', 'cm_vehicles'], inplace=True)

In [83]:
# convert columns to lower case
avia.columns = [col.lower() for col in avia.columns]

In [84]:
avia.head()

,cm_mkey,cm_hassafetyrec,cm_highestinjury,cm_isstudy,cm_mode,cm_ntsbnum,cm_originalpublisheddate,cm_mostrecentreporttype,cm_probablecause,cm_latitude,cm_longitude,cm_city,cm_country,cm_eventdate,cm_state,cm_agency,cm_boardlaunch,cm_eventtype,cm_launch,cm_reportdate,cm_reporttype,airportid,airportname,analysisnarrative,factualnarrative,cm_fatalinjurycount,cm_minorinjurycount,cm_seriousinjurycount,accidentsitecondition,cm_docketoriginalpublishdate,cm_vehiclenum,damagelevel,explosiontype,firetype,serialnumber,aircraftcategory,amateurbuilt,make,model,numberofengines,registrationnumber,gaflight,cm_engines,airmedical,airmedicaltype,flightoperationtype,flightscheduledtype,flightservicetype,flightterminaltype,operatorname,registeredowner,regulationflightconductedunder,revenuesightseeing,secondpilotpresent,cm_events,cm_findings
0,56265,False,Serious,False,Aviation,ATL03FA030,2004-09-01T04:00:00Z,Final,The pilot's inadequate preflight and in-flight...,33.811389,-84.606941,Austell,USA,2002-12-31T18:36:00Z,GA,NTSB,False,ACC,Yes,2004-09-01T04:00:00Z,DirectorBrief,FTY,Fulton County Airport,"According to air traffic control records, the ...",HISTORY OF FLIGHT&#x0D;\n&#x0D;\nOn December 3...,0.0,0.0,2.0,VMC,NaN,1,Substantial,None,None,28-4698,AIR,False,Piper,PA-28-180,1.0,N5118G,True,"[{'cm_engineNum': 1, 'engineType': 'REC'}]",False,None,PERS,None,None,None,None,U.S. Army Fort Meade Flying Activity,091,False,False,NaN,NaN
1,56266,False,None,False,Aviation,NYC03LA034,2003-06-25T04:00:00Z,Final,The pilot's delay in aborting the takeoff. A ...,40.318054,-77.876113,Mount Union,USA,2002-12-31T13:00:00Z,PA,NTSB,False,ACC,None,2003-06-25T04:00:00Z,DirectorBrief,MUU,Huntington County Airport,The pilot stated that he was departing on runw...,"On December 31, 2002, about 1200 eastern stand...",0.0,0.0,0.0,VMC,NaN,1,Substantial,None,None,1526,AIR,False,SOCATA,TB 9,1.0,N136ER,True,"[{'cm_engineNum': 1, 'engineType': 'REC'}]",False,None,PERS,None,None,None,None,Steven Nagorny,091,False,False,NaN,NaN
2,56278,False,None,False,Aviation,ATL03LA028,2004-03-02T05:00:00Z,Final,The pilot's inflight encounter with wind shear...,27.236665,-80.561668,Stuart,USA,2002-12-31T12:30:00Z,FL,NTSB,False,ACC,None,2004-03-02T05:00:00Z,DirectorBrief,64FA,Naked Lady Ranch (PVT),The student pilot conducted a visual approach ...,"On December 31, 2002, at 1130 eastern standard...",0.0,0.0,0.0,VMC,NaN,1,Substantial,None,None,17252865,AIR,False,Cessna,172F,1.0,N5338R,True,"[{'cm_engineNum': 1, 'engineType': 'REC'}]",False,None,PERS,None,None,None,Christopher S. Flom,"Aero Trade, Inc.",091,False,False,NaN,NaN
3,56272,False,None,False,Aviation,FTW03LA072,2004-06-02T04:00:00Z,Final,The pilot's attempt to takeoff during known ad...,28.266666,-94.050003,High Island 471,USA,2002-12-31T10:15:00Z,None,NTSB,False,ACC,None,2004-06-02T04:00:00Z,DirectorBrief,None,None,While repositioning the helicopter on an offsh...,"On December 31, 2002, approximately 0915 centr...",0.0,0.0,0.0,VMC,NaN,1,Substantial,None,None,52157,HELI,False,Bell,206-L4,1.0,N177AL,True,"[{'cm_engineNum': 1, 'engineType': 'TS'}]",False,None,POSI,None,None,None,None,Air Logistics LLC,091,False,False,NaN,NaN
4,56263,False,None,False,Aviation,FTW03LA070,2004-06-02T04:00:00Z,Final,The loss of engine power due to the separation...,25.920444,-97.480140,Brownsville,USA,2002-12-30T15:40:00Z,TX,NTSB,False,ACC,None,2004-06-02T04:00:00Z,DirectorBrief,BRO,Brownsville/South Padre Island,"The 2,900-hour pilot stated that the airplane ...","On December 30, 2002, at 1440 central standard...",0.0,0.0,0.0,VMC,NaN,1,Substantial,None,None,20700199,AIR,False,Cessna,T207,1.0,N1599U,True,"[{'cm_engineNum': 1, 'engineType': 'REC'}]",False,None,PERS,None,None,None,Alexandru Neacsu,Comercializadora Cano Inc.,091,False,False,NaN,NaN


In [31]:
show_null_counts(avia)

,null_ct
airmedicaltype,28453
flightservicetype,26702
flightterminaltype,26643
flightscheduledtype,26617
cm_launch,18486
operatorname,15268
cm_findings,10702
cm_events,10689
cm_docketoriginalpublishdate,10391
airportid,9081


In [7]:
avia.to_pickle('./data/cm_vehicles_flattened_joined')

NameError: name 'avia' is not defined

# EDA

In [ ]:
# From the saved pickled file from the robin_stopa.ipynb, is opened up here for further EDA

unpickled = pd.read_pickle('./cm_vehicles_flattened_joined')


In [ ]:
# Setting this option to see all columns in pickle file

pd.set_option('display.max_columns', 500)

In [ ]:
unpickled

In [ ]:
#This here helped lower case all the 'make' in the dataset.

unpickled['make'] = unpickled['make'].str.lower()

In [ ]:
unpickled.head(10)

In [ ]:
# This here allows us to look at the count per all aircraft in this dataset.  

unpickled['aircraftCategory'].value_counts()

In [ ]:
unpickled['aircraftCategory'].value_counts().plot(kind='bar')

# This graph below shows the what type of aircarft is most common in this dataset
# Based off this majority of the dataset is airplanes

This graph here gives better insight on the dataset we are working with.  This clearly shows that there are much more
aircraft in this dataset than any other type of air vehicle. From this our group wanted to only work with air craft since it
is the majority of the dataset. 

In [ ]:
# Saving a variable to this value count of 'make' of aircraft to plot this on a graph


var = unpickled['make'].value_counts().head(10)
var

In [ ]:
# top 10 most popular makes of vehicles in this dataset
# 1. Cessna
# 2. Piper
# 3. beech

var.plot(kind='bar')

This graph here shows the common 'make' of aircraft in the dataset.  Off the bad we can see the majorirty of dataset are 
private owned airplanes, not commercial. From this our group decided we only wanted to work with commercial airplanes (boeing, etc.)
as that information is more relevant to the population.

In [ ]:
# Gives a count per all the different answeres/outputs in the highestinjury column.  It shows most are none.
unpickled['cm_highestinjury'].value_counts()

In [ ]:
plot_injury = unpickled['cm_highestinjury'].value_counts()

plot_injury.plot(kind='bar', xlabel = 'Severity of Injury', ylabel = 'Count of Severity', title = 'The Count per Serverity of Injury')

#This plot looks at the highestinjury column and counts, which injuries were the most common in this dataset.


The graph here above gives a count per all the type of severity recorded in this pickled dataset.  This here clearly shows 
that most accidents have not injury.  However it should not dismiss the other data shown such as fatal injury around
5000.  This graph gives insight on the types of accidents that occured in this dataset.  

In [ ]:
# This code here allows me to look at the top ten make's of planes that have the highest count of accidents that have a 
# fatal injury severity in the highestinjury column.  I wanted to see what makes of plane were having the most fatal 
# accidents

unpickled[unpickled.cm_highestinjury == "Fatal"].make.str.lower().value_counts().head(10).index

In [ ]:
#This here shows me the top 10 most popular planes

unpickled.make.value_counts().head(10)

In [ ]:
# Proportions of fatal accident per make plans.  The numerator is the number of times this make of airplane was in a fatal 
# accident. The denominator is the total number of that make of airplane in the dataset.  
# This here will give us proportional data of what makes are most fatal based on how many is recorded in the dataset.
# This gives a better understanding on what makes could be more fatal.


Ces = (1066/7588) * 100
Pip = (773/4288) * 100
Bee = (445/1624) * 100
Bel = (123/730) * 100
Moo = (112/425) * 100
Rob = (60/429) * 100
Cir = (58/170) * 100
Nor = (44/131) * 100
Bella = (39/262) * 100
Hug = (35/243) * 100


In [ ]:
df_2 = pd.DataFrame({'Make of Plane':['Cessna', 'Piper', 'Beech', 'Bell', 'Mooney', 'Robinson', 'Cirrus design corp',
                           'North American', 'Bellanca', 'Hughes'], 'Proportion Fatal (%)':[Ces, Pip, Bee, Bel, Moo,
                                                                          Rob, Cir, Nor, Bella, Hug]})
ax = df_2.plot.bar(x='Make of Plane', y='Proportion Fatal (%)', rot=90, title = 'Proportion of Fatal Accidents per Make')

This graph shows the proportions of fatal accidents per make of airplane. Since the previous EDA showed that Cessna planes 
were much more common, it was essential to find the proportion of airplanes and compare it the percent of fatality.
This shows that Cirrus Desich Corp and North American, proportionally, have the most fatal accidents in this dataset.  